# Classical Time-Series Modelling Methods

Exploring machine learning time-series methods using StatsModels library and randomly generated data.

### Autoregression 

An Autoregression model predicts the next step in a time-series set of data, based on previous steps. 

In [4]:
from statsmodels.tsa.ar_model import AutoReg
from random import random

#importing random dataset
data = [x + random() for x in range(1,500)]

#fitting model
AR = AutoReg(data, lags=1)
AR_model_fit = AR.fit()

#predictions
yhat = AR_model_fit.predict(501, 1000)
print(yhat)

[500.48080796 501.47748714 502.47414935 503.47079458 504.46742284
 505.46403413 506.46062844 507.45720578 508.45376615 509.45030955
 510.44683597 511.44334542 512.4398379  513.43631341 514.43277195
 515.42921352 516.42563812 517.42204574 518.4184364  519.41481009
 520.41116681 521.40750656 522.40382934 523.40013515 524.396424
 525.39269588 526.38895079 527.38518873 528.38140971 529.37761372
 530.37380076 531.36997084 532.36612396 533.3622601  534.35837928
 535.3544815  536.35056675 537.34663504 538.34268637 539.33872073
 540.33473813 541.33073856 542.32672203 543.32268854 544.31863809
 545.31457068 546.3104863  547.30638496 548.30226666 549.29813141
 550.29397919 551.28981001 552.28562387 553.28142077 554.27720071
 555.27296369 556.26870972 557.26443878 558.26015089 559.25584604
 560.25152423 561.24718546 562.24282974 563.23845706 564.23406743
 565.22966084 566.22523729 567.22079678 568.21633933 569.21186491
 570.20737355 571.20286522 572.19833995 573.19379772 574.18923853
 575.1846624